## This Notebook will create the database for the temporary project washing machine manuals

The intention of this notebook is to create a clean and best practice database structure, along with utilizing snowflakes AI functions.

The intended database structure is as follows: 

- **documents** (Stores metadata about each manual)  
  - `document_id` (Unique ID for each manual)  
  - `doc_name` (Document name)
  - `version` (Version or revision number)  
  - `relative_path` (Original PDF file path or S3 URL) 
  - `stage_name`  (snowflake stage name (source))
  - `size`  (size in bytes of the PDF document) 

<!-- - **machine_types** (Stores metadata about each manual)  
  - `machine_type_id` (Unique ID for each machine type)
  - `manual_id` (foreign key referencing `manuals`)  
  - `name` (Type of machine or equipment) -->

- **sections** (Defines logical sections and subsections within each manual)  
  - `section_id` (Unique ID for the section)  
  - `manual_id` (Foreign key referencing `manuals`)  
  - `title` (Title or heading of the section)  
  - `order_num` (Numerical order of the section in the manual)  
  - `parent_section_id` (Optional FK for nested subsections)  

- **chunks small** (1024 characters, 64 overlap)
  - `chunk_id` (Unique ID for the chunk)  
  - `section_id` (Foreign key referencing `sections`)  
  - `chunk_text` (The text content of the chunk)  
  - `chunk_order` (Order of the chunk within the section)  
  - `embedding` (Vector for semantic search or embeddings)  

- **chunks large** (4096 characters, overlap 256)
  - `chunk_id` (Unique ID for the chunk)  
  - `section_id` (Foreign key referencing `sections`)  
  - `chunk_text` (The text content of the chunk)  
  - `chunk_order` (Order of the chunk within the section)  
  - `embedding` (Vector for semantic search or embeddings)  

- **images** (Stores references to images extracted from the manual)  
  - `image_id` (Unique ID for the image)  
  - `manual_id` (Foreign key referencing `manuals`)  
  - `page_number` 
  - `section_id` (Foreign key referencing `sections`)  
  - `order_num` (Display order within the section)  
  - `image_path` (S3 or web-accessible path to the image)  
  - `description`   



In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keyring
import os 
import snowflake.connector as sf_connector # ( https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-connect)
from snowflake.connector.pandas_tools import write_pandas # (https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-api#write_pandas)
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_docling import DoclingLoader
from collections import defaultdict
from PIL import Image
from io import BytesIO
import cv2
import numpy as np
from tqdm import tqdm
import time
import re
import json

# Set max rows to display in pandas DataFrame 200
pd.set_option('display.max_rows', 200)

In [ ]:
account_identifier = keyring.get_password('NC_Snowflake_Trial_Account_Name', 'account_identifier')
user_name = "EMHALDEMO1"
password = keyring.get_password('NC_Snowflake_Trial_User_Password', user_name)
database = "WASHING_MACHINE_MANUALS"
schema = "PUBLIC"

print("Account Identifier: ", account_identifier)
print("User Name: ", user_name)
print("Database: ", database)
print("Schema: ", schema)

try:
    connection_parameters = {
        "account_identifier": account_identifier,
        "user": user_name,
        "password": password,
        "role": "ACCOUNTADMIN",
        "warehouse": "COMPUTE_WH",
        "database": database,
        "schema": schema
    }
except:
        connection_parameters = {
        "account_identifier": account_identifier,
        "user": user_name,
        "password": password,
        "role": "ACCOUNTADMIN",
        "warehouse": "COMPUTE_WH",
        "database": "SNOWFLAKE",
        "schema": "CORTEX"
    }

# Connect to Snowflake
conn = sf_connector.connect(
    user=connection_parameters['user'],
    password=connection_parameters['password'],
    account=connection_parameters['account_identifier'],
    warehouse=connection_parameters['warehouse'],
    database=connection_parameters['database'],
    schema=connection_parameters['schema'],
    role=connection_parameters['role']
)

cursor = conn.cursor()
cursor.execute(f" CREATE DATABASE IF NOT EXISTS {database}; ")
cursor.execute(f" CREATE SCHEMA IF NOT EXISTS {database}.{schema}; ")
cursor.execute(f" USE DATABASE {database}; ")
cursor.execute(f" USE SCHEMA {schema}; ")



Account Identifier:  EPTJRCA-HWB83214
User Name:  EMHALDEMO1
Database:  WASHING_MACHINE_MANUALS
Schema:  PUBLIC


## Create a stage for the PDF files with the code below
#### DO NOT RUN - unless you don't have the documents in the stage.

In [ ]:
# Creating stage to dump PDF documents into
# cursor.execute(" create or replace stage docs ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') DIRECTORY = ( ENABLE = true ); ")

## Creating documents table

In [66]:
cursor.execute("""
    CREATE OR REPLACE TABLE DOCUMENTS (
    DOCUMENT_ID INT AUTOINCREMENT PRIMARY KEY,
    DOCUMENT_NAME STRING,
    DOC_VERSION STRING,
    RELATIVE_PATH STRING NOT NULL,
    SIZE NUMBER,
    STAGE_NAME STRING DEFAULT '@docs',
    CREATED_AT TIMESTAMP_LTZ DEFAULT CURRENT_TIMESTAMP()
);
""")

cursor.execute("""
    INSERT INTO DOCUMENTS (DOCUMENT_NAME, SIZE, RELATIVE_PATH)
    SELECT relative_path, size, file_url
    FROM DIRECTORY(@docs);
""")



In [67]:
# Lets see the table
cursor.execute("""
    SELECT * 
    FROM DOCUMENTS;
""")

documents_df = cursor.fetch_pandas_all()
documents_df.head()

,DOCUMENT_ID,DOCUMENT_NAME,DOC_VERSION,RELATIVE_PATH,SIZE,STAGE_NAME,CREATED_AT
0,1,WGG254Z0GB.pdf,None,https://bbb84589.snowflakecomputing.com/api/fi...,3291555,@docs,2025-04-16 05:35:39.026000-07:00
1,2,technical-manual-w11663204-revb.pdf,None,https://bbb84589.snowflakecomputing.com/api/fi...,17270389,@docs,2025-04-16 05:35:39.026000-07:00


# The section below focuses on creating chunks_large and chunks_small.

Different size chunks are good at different things - it could be a good idea to store both size, especially during testing

In [68]:
# Shows local file placement

pdf_files_path = ".\\Washer_Manuals"
pdf_file_list = [doc for doc in documents_df["DOCUMENT_NAME"] if doc.endswith(".pdf")]

for idx,filename in enumerate(pdf_file_list):
    if filename.endswith(".pdf"):
        file_path = os.path.join(pdf_files_path, filename)
        print(f"Document number: {idx}  : ",file_path)
    else:
        continue

Document number: 0  :  .\Washer_Manuals\WGG254Z0GB.pdf
Document number: 1  :  .\Washer_Manuals\technical-manual-w11663204-revb.pdf


## Creating chunks tables with vector embeddings

To include page numbers, i decided to create the tables using pandas, and then uploading them to snowflake

Followed by that will be a query to crete a vector embeddings.

In [ ]:
## Extracting section headers from the PDF files

def extract_text_chunks(file_path, manual_id, chunk_size=512, chunk_overlap=128):
    loader = PDFPlumberLoader(file_path)
    docs = loader.load()

    # Step 1: Combine all text across pages with page tracking
    all_text = ""
    page_map = []  # (char_index, page_number)

    for doc_page in docs:
        text = doc_page.page_content.strip().replace('\n', ' ')
        start_idx = len(all_text)
        all_text += text + " "  # Add space to separate pages
        end_idx = len(all_text)
        page_map.append((start_idx, end_idx, doc_page.metadata['page']))

    # Step 2: Create chunks with overlap, spanning across pages
    chunks = []
    chunk_order = []
    page_start_list = []
    page_end_list = []

    idx = 0
    chunk_idx = 0

    while idx < len(all_text):
        chunk = all_text[idx:idx + chunk_size]

        # Determine pages involved in this chunk
        chunk_start = idx
        chunk_end = idx + len(chunk)

        pages_in_chunk = [
            page_num
            for start, end, page_num in page_map
            if not (end <= chunk_start or start >= chunk_end)  # overlap condition
        ]

        page_start = min(pages_in_chunk) if pages_in_chunk else None
        page_end = max(pages_in_chunk) if pages_in_chunk else None

        chunks.append(chunk)
        page_start_list.append(page_start)
        page_end_list.append(page_end)
        chunk_order.append(chunk_idx)

        chunk_idx += 1
        idx += chunk_size - chunk_overlap

    # Step 3: Create DataFrame
    rows = [{
        'DOCUMENT_ID': manual_id,
        'PAGE_START_NUMBER': start,
        'PAGE_END_NUMBER': end,
        'CHUNK_TEXT': chunk,
        'CHUNK_ORDER': order
    } for chunk, start, end, order in zip(chunks, page_start_list, page_end_list, chunk_order)]

    df = pd.DataFrame(rows, columns=["DOCUMENT_ID", "PAGE_START_NUMBER", "PAGE_END_NUMBER", "CHUNK_TEXT", "CHUNK_ORDER"])
    return df


large_chunks_df = pd.DataFrame()
for row in tqdm(documents_df.iterrows(), total = len(documents_df)):
    manual_id = row[1]["DOCUMENT_ID"]
    file_path = os.path.join(pdf_files_path, row[1]["DOCUMENT_NAME"])

    tmp_chunked_df = extract_text_chunks(file_path = file_path, 
                        manual_id = manual_id,
                        chunk_size = 6000,#1024,
                        chunk_overlap = 128)  # Show first 5 chunks
    large_chunks_df = pd.concat([large_chunks_df, tmp_chunked_df], ignore_index=True)

large_chunks_df

100%|██████████| 2/2 [00:17<00:00,  8.88s/it]


,DOCUMENT_ID,PAGE_START_NUMBER,PAGE_END_NUMBER,CHUNK_TEXT,CHUNK_ORDER
0,1,0,5,Register your b M o ge s y n c t B e h f o r w...,0
1,1,5,10,"ectly, this causes a tripping hazard. ▶ Lay ho...",1
2,1,10,17,components by type and dispose of them separ- ...,2
3,1,16,24,"Off to switch off the appliance. → ""Basic ope...",3
4,1,24,30,"has been reached, it remains constant througho...",4
5,1,30,37,Start / Reload. The appliance pauses. 13.12 U...,5
6,1,37,42,because the laundry is unevenly distrib- uted....,6
7,1,42,46,ault Cause and troubleshooting The spin progra...,7
8,1,46,51,er (FD) 2012/19/EU concerning on the appliance...,8
9,2,0,6,TECHNICAL MANUAL Maytag® 3.5 cu ft Commercial-...,0


In [76]:
create_table_sql = """
CREATE OR REPLACE TABLE CHUNKS_LARGE (
    CHUNK_ID INT AUTOINCREMENT PRIMARY KEY,
    DOCUMENT_ID INT NOT NULL,
    PAGE_START_NUMBER INT,
    PAGE_END_NUMBER INT,
    CHUNK_ORDER INT,
    CHUNK_TEXT STRING NOT NULL,
    EMBEDDING VECTOR(FLOAT, 1024),
    CREATED_AT TIMESTAMP_LTZ DEFAULT CURRENT_TIMESTAMP(),
    CONSTRAINT fk_document
        FOREIGN KEY (DOCUMENT_ID)
        REFERENCES DOCUMENTS(DOCUMENT_ID)
);
"""
cursor.execute(create_table_sql)

In [77]:
success, nchunks, nrows, output = write_pandas(
    conn=conn,
    df=large_chunks_df,
    database =database,
    table_name="CHUNKS_LARGE",
    schema=schema,
    auto_create_table=False,
    overwrite=False
)

print(f"Success: {success}, Chunks: {nchunks}, Rows: {nrows}")

# Update the embeddings for the chunks in the CHUNKS_LARGE table
cursor.execute("""
    UPDATE CHUNKS_LARGE
    SET EMBEDDING = SNOWFLAKE.CORTEX.EMBED_TEXT_1024(
        'snowflake-arctic-embed-l-v2.0',
        CHUNK_TEXT
    )
    WHERE EMBEDDING IS NULL;
""")


Success: True, Chunks: 1, Rows: 23


In [74]:
small_chunks_df = pd.DataFrame()
for row in tqdm(documents_df.iterrows(), total = len(documents_df)):
    manual_id = row[1]["DOCUMENT_ID"]
    tmp_chunked_df = extract_text_chunks(file_path = file_path, 
                        manual_id = manual_id,
                        chunk_size = 1024,
                        chunk_overlap = 64)  # Show first 5 chunks
    small_chunks_df = pd.concat([small_chunks_df, tmp_chunked_df], ignore_index=True)


create_table_sql = """
CREATE OR REPLACE TABLE CHUNKS_SMALL (
    CHUNK_ID INT AUTOINCREMENT PRIMARY KEY,
    DOCUMENT_ID INT NOT NULL,
    PAGE_START_NUMBER INT,
    PAGE_END_NUMBER INT,
    CHUNK_ORDER INT,
    CHUNK_TEXT STRING NOT NULL,
    EMBEDDING VECTOR(FLOAT, 1024),
    CREATED_AT TIMESTAMP_LTZ DEFAULT CURRENT_TIMESTAMP(),
    CONSTRAINT fk_document
        FOREIGN KEY (DOCUMENT_ID)
        REFERENCES DOCUMENTS(DOCUMENT_ID)
);
"""
cursor.execute(create_table_sql)

success, nchunks, nrows, output = write_pandas(
    conn=conn,
    df=small_chunks_df,
    database =database,
    table_name="CHUNKS_SMALL",
    schema=schema,
    auto_create_table=False,
    overwrite=False
)

print(f"Success: {success}, Chunks: {nchunks}, Rows: {nrows}")

# Update the embeddings for the small chunks
cursor.execute("""
    UPDATE CHUNKS_SMALL
    SET EMBEDDING = SNOWFLAKE.CORTEX.EMBED_TEXT_1024(
        'snowflake-arctic-embed-l-v2.0',
        CHUNK_TEXT
    )
    WHERE EMBEDDING IS NULL;
""")


100%|██████████| 2/2 [00:23<00:00, 11.67s/it]


Success: True, Chunks: 1, Rows: 230


## Creating sections table using LLM for TOC extraction

In [ ]:
def extract_TOC(text: str, model : str) -> str:
    prompt = (
    """
    I will provide a long string of text that most likely contains a table of contents, 
    although it may also include additional body text from a document. Your task is to carefully 
    extract only the table of contents and structure it as a JSON object in the following 
    format:
    {
      "Section": "<section name>",
      "Section Number": "<section name>",
      "Page": <page number>,
      "Sub Sections" : [{
        "Section": "<section name>",
        "Section Number": "<section name>",
        "Page": <page number>,
        "Sub Sections" : []}
      ],
    }    

    Guideines:
    - All keys in the json object must be either "Section", "Section Number", "Page", "Sub Sections".
    - "Section Number" must be represented as an integer or float - E.G: 1, 2, 5.3, 1,4, etc.
    - Ignore any text that is not part of the table of contents.
    - Ensure that sub-sections are nested appropriately under their parent section.
    - Page numbers should be extracted as integers, if possible.
    - Be tolerant of inconsistencies in formatting, spacing, or punctuation (e.g. dashes, colons, ellipses).
    - Do not include duplicate or repeated sections.
    - You should only consider items which are part of the table of contents, nothing before, nothing after.
    - "Section" must consist of words
    - "Section Number" must be represented as an integer or float - E.G: 1, 2, 5.3, 1,4, etc.
    - You must include a top level key value pair called "Section":"Table of contents".

    """
    f"Text:\n{text}"
    )
    start_time = time.time()
    result = cursor.execute(f"""
        SELECT SNOWFLAKE.CORTEX.COMPLETE('{model}', $$ {prompt} $$)
    """)
    print(f"Runtime in seconds: {time.time() - start_time:.4f}")

    return cursor.fetch_pandas_all().iloc[0,0]


# This example prints out section 4 of the first document of the database. mistral-large2 mistral-7b
# llm_output = extract_TOC(df_large_chunks.loc[0,"CHUNK"], model = 'mistral-7b')
llm_output = extract_TOC(large_chunks_df.loc[0,"CHUNK_TEXT"], model = 'llama3.1-70b')
llm_output

In [ ]:
def extract_json_from_llm_output(llm_output: str) -> dict:
    try:
        # Confirming that a JSON block is returned
        match = re.search(r"```\s*(\{.*?\})\s*```", llm_output, re.DOTALL)
        if not match:
            raise ValueError("No JSON code block found in the text.")

        # Extracting sub string (json string)
        raw_json = match.group(1)

        # Clean common JSON errors (e.g., trailing commas)
        cleaned_json = re.sub(r",\s*([\]}])", r"\1", raw_json)  # remove trailing commas before ] or }
        
        # Parse string to json
        parsed = json.loads(cleaned_json)
        return parsed
    
    except Exception as e:
        print("Failed to extract JSON:", e)
        return {}

        
parsed_dict = extract_json_from_llm_output(llm_output)
print(json.dumps(parsed_dict, indent=2))

{
  "Section": "Table of contents",
  "Section Number": "",
  "Page": "",
  "Sub Sections": []
}


In [ ]:
-- Creates the table for storing the LARGE chunks and vector embeddings
CREATE OR REPLACE TABLE CHUNKS_LARGE (
    CHUNK_ID INT AUTOINCREMENT PRIMARY KEY,
    DOCUMENT_ID INT NOT NULL,
    CHUNK_INDEX INT,
    CHUNK STRING NOT NULL,
    EMBEDDING VECTOR(FLOAT, 1024),
    CREATED_AT TIMESTAMP_LTZ DEFAULT CURRENT_TIMESTAMP(),
    CONSTRAINT fk_document
        FOREIGN KEY (DOCUMENT_ID)
        REFERENCES DOCUMENTS(DOCUMENT_ID)
);


-- Creates a temp table with parsed text (1 row for each document, with a super long string of raw text of the document)
CREATE OR REPLACE TEMP TABLE parsed_text_table AS
SELECT 
  relative_path as document_name,
  size,
  file_url,
  BUILD_SCOPED_FILE_URL(@docs, relative_path) AS scoped_file_url,
  TO_VARCHAR(
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT(@docs, relative_path, {'mode': 'OCR'}) -- trying this instead of LAYOUT
  ) AS full_text
FROM DIRECTORY(@docs);


-- Temp table for extracting the content. (Disabled as it gave worse results when finding sections)
-- CREATE OR REPLACE TEMP TABLE parsed_text_table_extracted AS
-- SELECT
--   document_name,
--   PARSE_JSON(full_text):content::STRING AS full_text
-- FROM parsed_text_table;


-- Using the temporary table to fill the CHUNKS tables with 
INSERT INTO CHUNKS_LARGE (DOCUMENT_ID, CHUNK_INDEX, CHUNK, EMBEDDING)
SELECT 
    d.DOCUMENT_ID,
    chunk_data.index AS CHUNK_INDEX,
    chunk_data.value::STRING AS CHUNK,
    SNOWFLAKE.CORTEX.EMBED_TEXT_1024('snowflake-arctic-embed-l-v2.0', chunk_data.value::STRING) AS EMBEDDING
FROM parsed_text_table p
JOIN DOCUMENTS d ON p.document_name = d.document_name
JOIN LATERAL FLATTEN(
    INPUT => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
        p.full_text,
        'none',         -- 'none' or 'markdown'
        6000,           -- chunk size
        256             -- overlap
    )
) AS chunk_data
WHERE p.full_text IS NOT NULL;

SELECT * 
FROM CHUNKS_LARGE 
LIMIT 30;

### And now, small chunks table

In [ ]:
-- Creates the table for storing the LARGE chunks and vector embeddings
CREATE OR REPLACE TABLE CHUNKS_SMALL (
    CHUNK_ID INT AUTOINCREMENT PRIMARY KEY,
    DOCUMENT_ID INT NOT NULL,
    CHUNK_INDEX INT,
    CHUNK STRING NOT NULL,
    EMBEDDING VECTOR(FLOAT, 1024),
    CREATED_AT TIMESTAMP_LTZ DEFAULT CURRENT_TIMESTAMP(),
    CONSTRAINT fk_document
        FOREIGN KEY (DOCUMENT_ID)
        REFERENCES DOCUMENTS(DOCUMENT_ID)
);

-- Creating a temp table is not needed since we made one with the large chunks.

-- Using the temporary table to fill the CHUNKS tables with 
INSERT INTO CHUNKS_SMALL (DOCUMENT_ID, CHUNK_INDEX, CHUNK, EMBEDDING)
SELECT 
    d.DOCUMENT_ID,
    chunk_data.index AS CHUNK_INDEX,
    chunk_data.value::STRING AS CHUNK,
    SNOWFLAKE.CORTEX.EMBED_TEXT_1024('snowflake-arctic-embed-l-v2.0', chunk_data.value::STRING) AS EMBEDDING
FROM parsed_text_table p
JOIN DOCUMENTS d ON p.document_name = d.document_name
JOIN LATERAL FLATTEN(
    INPUT => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
        p.full_text,
        'none',    -- 'none' or 'markdown'
        512,           -- chunk size
        64             -- overlap
    )
) AS chunk_data
WHERE p.full_text IS NOT NULL;

SELECT * 
FROM CHUNKS_SMALL 
LIMIT 10;

### This section will create the sections table, using an LLM to extract the TOC. 

This method assumes that the TOC is found in the first LARGE chunk for each document.
While this is absolutely not best practice, alternative methods were shown to be time consuming and inconsistent. For the sake of this task, this will be more than sufficient.

In [ ]:
CREATE OR REPLACE TABLE SECTIONS (
    SECTION_ID INT,
    DOCUMENT_ID INT NOT NULL,
    SECTION_NAME STRING,
    SECTION_NUMBER STRING,
    PAGE INT,
    PARENT_SECTION_ID INT,
    PARENT_SECTION_NUMBER STRING,
    CHUNK_ID INT,
    CREATED_AT TIMESTAMP_LTZ DEFAULT CURRENT_TIMESTAMP(),
    
    -- Constraints
    PRIMARY KEY (SECTION_ID),
    FOREIGN KEY (DOCUMENT_ID) REFERENCES DOCUMENTS(DOCUMENT_ID)
);

In [ ]:
import pandas as pd
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col
import time
import re
import json


session = Session.builder.getOrCreate()
df_large_chunks = session.table("CHUNKS_LARGE").to_pandas()
df_large_chunks.head()

In [ ]:
def extract_TOC(text: str, model : str) -> str:
    prompt = (
    """
    I will provide a long string of text that most likely contains a table of contents, 
    although it may also include additional body text from a document. Your task is to carefully 
    extract only the table of contents and structure it as a JSON object in the following 
    format:
    {
      "Section": "<section name>",
      "Section Number": "<section name>",
      "Page": <page number>,
      "Sub Sections" : [{
        "Section": "<section name>",
        "Section Number": "<section name>",
        "Page": <page number>,
        "Sub Sections" : []}
      ],
    }    

    Guideines:
    - All keys in the json object must be either "Section", "Section Number", "Page", "Sub Sections".
    - "Section Number" must be represented as an integer or float - E.G: 1, 2, 5.3, 1,4, etc.
    - Ignore any text that is not part of the table of contents.
    - Ensure that sub-sections are nested appropriately under their parent section.
    - Page numbers should be extracted as integers, if possible.
    - Be tolerant of inconsistencies in formatting, spacing, or punctuation (e.g. dashes, colons, ellipses).
    - Do not include duplicate or repeated sections.
    - You should only consider items which are part of the table of contents, nothing before, nothing after.
    - "Section" must consist of words
    - "Section Number" must be represented as an integer or float - E.G: 1, 2, 5.3, 1,4, etc.
    - You must include a top level key value pair called "Section":"Table of contents".

    """
    f"Text:\n{text}"
    )
    start_time = time.time()
    result = session.sql(f"""
        SELECT SNOWFLAKE.CORTEX.COMPLETE('{model}', $$ {prompt} $$)
    """).collect()
    print(f"Runtime in seconds: {time.time() - start_time:.4f}")
    
    return result


# This example prints out section 4 of the first document of the database. mistral-large2 mistral-7b
# llm_output = extract_TOC(df_large_chunks.loc[0,"CHUNK"], model = 'mistral-7b')
llm_output = extract_TOC(df_large_chunks.loc[0,"CHUNK"], model = 'llama3.1-70b')
llm_output

In [ ]:
def extract_json_from_llm_output(llm_output: str) -> dict:
    try:
        # Confirming that a JSON block is returned
        match = re.search(r"```\s*(\{.*?\})\s*```", llm_output, re.DOTALL)
        if not match:
            raise ValueError("No JSON code block found in the text.")

        # Extracting sub string (json string)
        raw_json = match.group(1)

        # Clean common JSON errors (e.g., trailing commas)
        cleaned_json = re.sub(r",\s*([\]}])", r"\1", raw_json)  # remove trailing commas before ] or }
        
        # Parse string to json
        parsed = json.loads(cleaned_json)
        return parsed
    
    except Exception as e:
        print("Failed to extract JSON:", e)
        return {}

        
parsed_dict = extract_json_from_llm_output(llm_output[0][0])
print(json.dumps(parsed_dict, indent=2))

In [ ]:
def create_pandas_table(parsed_dict: dict, document_id: int) -> pd.DataFrame:
    rows = []
    section_id_counter = [1]  # counter for Section_IDs

    def recurse(sections, parent_id=None, parent_number=None):
        for section in sections:
            current_id = section_id_counter[0]
            section_id_counter[0] += 1

            current_number = section.get("Section Number")

            rows.append({
                "Section_ID": current_id,
                "Document_ID": document_id,
                "Section_Name": section.get("Section"),
                "Section_Number": current_number,
                "Page": section.get("Page"),
                "Parent_Section_ID": parent_id,
                "Parent_Section_Number": parent_number
            })

            # Recurse into sub-sections
            sub_sections = section.get("Sub Sections", [])
            if isinstance(sub_sections, list) and sub_sections:
                recurse(
                    sub_sections,
                    parent_id=current_id,
                    parent_number=current_number
                )

    top_level_sections = parsed_dict.get("Sub Sections", []) # Some quirky issue with the format of the returned JSON object.
    # top_level_sections = parsed_dict["Table of contents"]
    recurse(top_level_sections)

    section_df = pd.DataFrame(rows)
    section_df["Chunk_ID"] = None

    return section_df

parsed_dict = extract_json_from_llm_output(llm_output[0][0])
document_id = df_large_chunks.loc[0,"DOCUMENT_ID"]
df_sections = create_pandas_table(parsed_dict, document_id)
df_sections.head(8)

In [ ]:
df_sections.columns = [col.upper() for col in df_sections.columns]

session.write_pandas(
    df_sections,
    table_name="SECTIONS",
    overwrite=False,
    auto_create_table=False
)

In [ ]:
SELECT * 
FROM SECTIONS 
LIMIT 10;